In [6]:
# Importing libraries
from netCDF4 import Dataset
from netCDF4 import MFDataset
import netCDF4 as nc
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.dates as mdates
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap, LogNorm
import re 
import pylag
from shapely.geometry import Point
import contextily as ctx
import datetime
import dask
import dask.dataframe as dd
import gc 
import concurrent.futures
import cftime  # Import cftime to handle cftime.DatetimeGregorian objects
from datetime import datetime

In [7]:
#define a path to the FVCOME data directory /home/samin/data/FVCOME_OUTPUT/Gldata/FVCOMEDATA
# go forlder 202301 and red nc file contain *2230101.nc
# define the path to the data directory
data_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output'
def sort_key(file):
    filename = os.path.basename(file)
    number = int(filename.split('__')[1].split('.')[0])
    return number
files = glob.glob(data_dir + "/Fvcome_huron_estuary_2023_Winter_*.nc")
files.sort(key=sort_key)

In [8]:
ds = xr.open_dataset(files[0])
ds

<xarray.Dataset>
Dimensions:                   (time: 121, particles: 4950)
Coordinates:
  * time                      (time) datetime64[ns] 2023-01-01 ... 2023-01-31
Dimensions without coordinates: particles
Data variables:
    group_id                  (particles) int32 ...
    longitude                 (time, particles) float32 ...
    latitude                  (time, particles) float32 ...
    depth                     (time, particles) float32 ...
    host_fvcom                (time, particles) int32 ...
    error_status              (time, particles) int32 ...
    in_domain                 (time, particles) int32 ...
    is_beached                (time, particles) int32 ...
    land_boundary_encounters  (time, particles) int32 ...
    thetao                    (time, particles) float32 ...
    so                        (time, particles) float32 ...
Attributes:
    title:    PyLag -- Plymouth Marine Laboratory

In [15]:
# Loop through each file, load it, create group_number as a string with leading zeros, and save the updated NetCDF file
for file in files:
    # Read the NetCDF file
    ds = xr.open_dataset(file)

    # Convert the NetCDF 'group_id' variable to a DataFrame for processing
    netcdf_df = ds['group_id'].to_dataframe().reset_index()

    # Step 1: Count occurrences of each unique group_id in the NetCDF file
    netcdf_df['group_number'] = netcdf_df.groupby('group_id').cumcount()

    # Step 2: Combine 'group_id' and 'group_number' with leading zeros
    # Pad group_id to 2 digits and group_number to 3 digits
    netcdf_df['group_number'] = netcdf_df.apply(lambda row: f"{int(row['group_id']):0}{row['group_number']:02}", axis=1)

    # Step 3: Add the updated group_number column back into the NetCDF dataset as a string or integer
    ds['group_number'] = (('particles'), netcdf_df['group_number'].values)

    # Step 4: Save the updated NetCDF file
    output_file_path = os.path.join(data_dir, f"updated_{os.path.basename(file)}")
    ds.to_netcdf(output_file_path)

    print(f"Updated NetCDF file saved: {output_file_path}")


Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jan__1.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Feb__2.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Mar__3.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Apr__4.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_May__5.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jun__6.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_July__7.nc
Updated NetCDF file saved: /home/abolmaa

In [25]:
# read the updated NetCDF file
updated_files = glob.glob(data_dir + "/updated_Fvcome_huron_estuary_2023_Winter_*.nc")
updated_files.sort(key=sort_key)
#ds = xr.open_mfdataset(updated_files, combine='by_coords')

# read the firs file
ds = xr.open_dataset(updated_files[0])
# prnt the updated group_number variable values
print(ds['group_number'].values)

['000' '001' '002' ... '14745' '14746' '14747']


In [32]:
# Convert the NetCDF 'group_id' and 'group_number' variables to a DataFrame for processing
netcdf_df = ds['group_id'].to_dataframe().reset_index()

# Ensure that 'group_number' is extracted and correctly added
netcdf_df['group_number'] = ds['group_number'].values

# Step 1: Convert 'group_number' to integer if necessary, and format as a 5-digit string with leading zeros
netcdf_df['group_number'] = netcdf_df['group_number'].apply(lambda x: f"{int(x):05}")

# Step 2: Print values for debugging
print("First few rows of netcdf_df:")
print(netcdf_df.head())

# Step 3: Select relevant columns for saving to CSV
netcdf_df = netcdf_df[['group_id', 'group_number']]

# Save the result as a CSV file
netcdf_df.to_csv(os.path.join(data_dir, 'group_id_group_number.csv'), index=False)

print("CSV file saved with formatted group numbers")



First few rows of netcdf_df:
   particles  group_id group_number
0          0         0        00000
1          1         0        00001
2          2         0        00002
3          3         0        00003
4          4         0        00004
CSV file saved with formatted group numbers
